<a href="https://colab.research.google.com/github/Vynogradova/homework/blob/main/Homework%22%D0%90%D0%BD%D0%B0%D0%BB%D1%96%D0%B7_%D0%90_%D0%92_%D1%82%D0%B5%D1%81%D1%82%D1%96%D0%B2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
from math import ceil
from scipy import stats

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [4]:
# Розрахунок розміру ефекту на основі наших очікуваних показників
effect_size = sms.proportion_effectsize (0.20,0.19)
effect_size

np.float64(0.025241594409087353)

In [7]:
 # Розрахунок необхідного розміру вибірки
required_n = sms.NormalIndPower().solve_power(
    effect_size,
    alpha=0.05,
    power=0.8,
    ratio=1
)
required_n = ceil(required_n)
print(required_n)

24638


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [8]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cookie_cats.csv')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [19]:
print(f"Пропущені значення (NaN):")
print(df.isna().sum())

Пропущені значення (NaN):
userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64


In [26]:
df['retention_7'].mean()

np.float64(0.1860648194347426)

In [27]:
retention_by_version = df.groupby('version')['retention_7'].mean()
print(retention_by_version)


version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64


На перший погляд, контрольна версія gate_30 має трохи краще утримання на 7-й день, ніж тестова.
Різниця невелика (~0.8%).

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [33]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

control = df[df['version'] == 'gate_30']
treatment = df[df['version'] == 'gate_40']

success_control = control['retention_7'].sum()
success_treatment = treatment['retention_7'].sum()

n_control = control.shape[0]
n_treatment = treatment.shape[0]

count = np.array([success_control, success_treatment])
nobs = np.array([n_control, n_treatment])

z_stat, p_value = proportions_ztest(count, nobs)

confint_control = proportion_confint(success_control, n_control, alpha=0.05, method='normal')
confint_treatment = proportion_confint(success_treatment, n_treatment, alpha=0.05, method='normal')

print(f"z statistic: {z_stat:.5f}")
print(f"p-value: {p_value:.5f}")
print(f"Довірчий інтервал 95% для групи control: [{confint_control[0]:.5f}, {confint_control[1]:.5f}]")
print(f"Довірчий інтервал 95% для групи treatment: [{confint_treatment[0]:.5f}, {confint_treatment[1]:.5f}]")


z statistic: 3.16436
p-value: 0.00155
Довірчий інтервал 95% для групи control: [0.18656, 0.19384]
Довірчий інтервал 95% для групи treatment: [0.17845, 0.18555]


1. p-value = 0.00155 < 0.05 - різниця між контрольним (gate_30) і тестовим (gate_40) варіантом статистично значуща. Це означає, що зміна воріт на рівні 40 днів трохи знизила утримання користувачів на 7-й день.

2. Інтервали не перетинаються ([0.18656, 0.19384] vs [0.17845, 0.18555]). Це підтверджує, що різниця між версіями реальна і не випадкова.

Висновок: контрольна версія gate_30 забезпечує кращі показники утримання на 7-й день, ніж тестова gate_40.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


In [35]:
pd.crosstab(df['version'], df['retention_7'])

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [36]:
crosstab = pd.crosstab(df['version'], df['retention_7'])

In [41]:
chi2, p, dof, expected = stats.chi2_contingency(crosstab)

print(f"χ² = {chi2:.3f}")
print(f"p-value = {p:.5f}")
print(f"Ступені свободи = {dof}")
print("Очікувані частоти:\n", np.round(expected, 2))

χ² = 9.959
p-value = 0.00160
Ступені свободи = 1
Очікувані частоти:
 [[36382.9  8317.1]
 [37025.1  8463.9]]


$$H_0: p = p_0$$
$$H_a: p \ne p_0$$

де $p$ і $p_0$ ймовірності 7-денного утримання для версій gate_30 і gate_40.

p-value = 0.0016 < 0.05, тобто ми відхиляємо нульову гіпотезу H₀.

Це означає, що існує статистично значуща різниця у 7-денному утриманні між версіями gate_30 та gate_40.

Практично: gate_30 показує трохи вищий retention (≈19.0%) ніж gate_40 (≈18.2%), але ефект невеликий.